In [ ]:
!pip install pyabsa

In [ ]:
import re

URL_PATTERN = re.compile(r'https?://[^ ]+')
MENTIONS_PATTERN = re.compile(r'@[^ ]+')
HASHTAGS_PATTERN = re.compile(r'#[^ ]+')
SPECIAL_CHARS_PATTERN = re.compile(r'[^A-Za-zÀ-ž ]')
MULTIPLE_SPACES_PATTERN = re.compile(' +')

USER_DATA_COLUMNS = ['id', 'created_at', 'description', 'updated_bio', 'translated_bio', 'location', 'location_city',
                     'location_state', 'location_country' 'public_metrics_followers_count',
                     'public_metrics_following_count', 'public_metrics_tweet_count', 'public_metrics_listed_count',
                     'verified', 'extracted_urls', 'extracted_mentions', 'extracted_hashtags', 'tweets_in_dataset',
                     'retweets_in_dataset']

TWEET_DATA_COLUMNS = ['id', 'author_id', 'text', 'updated_text', 'created_at', 'lang', 'public_metrics_retweet_count',
                      'public_metrics_reply_count', 'public_metrics_like_count', 'public_metrics_quote_count',
                      'extracted_urls', 'extracted_mentions', 'extracted_hashtags']

TWEET_USER_MAP_COLUMNS = ['id', 'author_id', 'created_at', 'is_retweet']


In [ ]:
import pandas as pd
import numpy as np
# from pyabsa import available_checkpoints, TaskCodeOption
# checkpoint_map = available_checkpoints(task_code=TaskCodeOption.Aspect_Term_Extraction_and_Classification, show_ckpts=True)

In [ ]:
from pyabsa import ATEPCCheckpointManager

aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='multilingual',
                                   auto_device=True  # False means load model on CPU
                                   )

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [ ]:
# You can inference from a list of setences or a DatasetItem from PyABSA 
examples = ['Das Personal war sehr unhöflich, aber das Essen war sehr lecker']
inference_source = examples
atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

[2023-02-22 17:19:21] (2.0.28) The results of aspect term extraction have been saved in /content/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-02-22 17:19:21] (2.0.28) Example 0: Das <Personal:Negative Confidence:0.9934371113777161> war sehr unhöflich , aber das <Essen:Positive Confidence:0.9978693723678589> war sehr lecker


/usr/local/lib/python3.8/dist-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:566: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  lcf_cdm_vec = torch.tensor(
/usr/local/lib/python3.8/dist-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:647: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Modified_Tweet_Data.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
temp = df[df['user_category'] == 'PostDoc']
temp = temp[temp['lang'] == 'de']
temp = list(df[df['updated_text'] != "__NA__"]['updated_text'])

In [ ]:
inference_source = temp
atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

In [ ]:
i = 0
for output in atepc_result:
    i += 1
    if i == 20:
        break
    if len(output['aspect']) == 0:
        print("Empty")
    print(output['aspect'])
    print(output['sentiment'])

In [ ]:
aspectsList = []
sentiList = []
for output in atepc_result:
    if len(output['aspect']) >= 0:
        aspectsList += output['aspect']
        sentiList += output['sentiment']

In [ ]:
!pip install pyabsa==1.16.27

In [ ]:
import pandas as pd
import numpy as np
from pyabsa import ATEPCCheckpointManager

aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='multilingual',
                                   auto_device=True  # False means load model on CPU
                                   )


In [ ]:
from google.colab import drive
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Modified_Tweet_Data.csv")

In [ ]:
# You can inference from a list of setences or a DatasetItem from PyABSA 
examples = ['Das Personal war sehr unhöflich, aber das Essen war sehr lecker']
atepc_result = aspect_extractor.extract_aspect(inference_source=examples,  #
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

The results of aspect term extraction have been saved in /content/atepc_inference.result.json
Example 0: Das <Personal:Negative Confidence:0.9996588230133057> war sehr unhöflich , aber das <Essen:Positive Confidence:0.999847412109375> war sehr lecker


/usr/local/lib/python3.8/dist-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:362: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  lcf_cdm_vec = torch.tensor([f.lcf_cdm_vec for f in infer_features], dtype=torch.float32)
/usr/local/lib/python3.8/dist-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:410: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]


In [ ]:
temp = df[df['user_category'] == "PostDoc"]
temp = temp[temp['lang'] == "en"]
temp = list(temp[temp['updated_text'] != "__NA__"]['updated_text'])
len(temp)

44797

In [ ]:
def absa(df, category, lang, aspect_extractor):

    temp = df[df['user_category'] == category]
    temp = temp[temp['lang'] == lang]
    temp = list(temp[temp['updated_text'] != "__NA__"]['updated_text'])

    atepc_result = aspect_extractor.extract_aspect(inference_source=temp,  #
                          pred_sentiment=True,
                          print_result = False
                          )
    aspectsList = []
    sentiList = []
    
    for output in atepc_result:
        if len(output['aspect']) >= 0:
            aspectsList += output['aspect']
            sentiList += output['sentiment']
    
    if len(aspectsList) == 0:
        print("No aspects extracted")
        return None
    
    df = pd.DataFrame({
        'Aspects': aspectsList,
        'Sentiment': sentiList,
        })
    
    df_1 = pd.get_dummies(data=df['Sentiment'])

    agg_dict = {'Aspects': ['count']}
    if len(df_1.columns.values) >= 1:
        for col in df_1.columns.values:
            agg_dict[col] = ['sum']

    df = pd.concat([df, df_1], axis=1)

    new_df = df.groupby('Aspects').agg(agg_dict).sort_values(('Aspects', 'count'), ascending=False)

    new_df.to_csv("drive/MyDrive/Colab Notebooks/dataset/" + category + "_" + lang + ".csv")

    return new_df

In [ ]:
absa(df, "PostDoc", "de", aspect_extractor)
absa(df, "PostDoc", "en", aspect_extractor)
# absa(df, "Professor", "de", aspect_extractor)
# absa(df, "Professor", "en", aspect_extractor)
# absa(df, "PhD Student", "de", aspect_extractor)
# absa(df, "PhD Student", "en", aspect_extractor)
# absa(df, "Others", "de", aspect_extractor)
# absa(df, "Others", "en", aspect_extractor)
# absa(df, "Unknown", "de", aspect_extractor)
# absa(df, "Unknown", "en", aspect_extractor)
# absa(df, "Lecturer", "de", aspect_extractor)
# absa(df, "Lecturer", "en", aspect_extractor)
# absa(df, "PD", "de", aspect_extractor)
# absa(df, "PD", "en", aspect_extractor)

  0%|          | 0/57 [00:00<?, ?it/s, classifying aspect sentiments...]/usr/local/lib/python3.8/dist-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:410: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]
100%|██████████| 57/57 [01:37<00:00,  1.71s/it, classifying aspect sentiments...]


The results of aspect term extraction have been saved in /content/atepc_inference.result.json


100%|██████████| 11/11 [00:17<00:00,  1.63s/it, classifying aspect sentiments...]


The results of aspect term extraction have been saved in /content/atepc_inference.result.json


,Aspects,Negative,Neutral,Positive
,count,sum,sum,sum
Aspects,,,,
system,69,58,6,5
job,32,13,16,3
staff,30,18,9,3
thread,27,2,3,22
position,27,6,19,2
...,...,...,...,...
hanin,1,1,0,0
gregor mendel,1,0,1,0


In [ ]:
absa(df, "Professor", "de", aspect_extractor)
absa(df, "Professor", "en", aspect_extractor)
absa(df, "PhD Student", "de", aspect_extractor)
absa(df, "PhD Student", "en", aspect_extractor)
absa(df, "Others", "de", aspect_extractor)
absa(df, "Others", "en", aspect_extractor)
absa(df, "Unknown", "de", aspect_extractor)
absa(df, "Unknown", "en", aspect_extractor)
absa(df, "Lecturer", "de", aspect_extractor)
absa(df, "Lecturer", "en", aspect_extractor)
absa(df, "PD", "de", aspect_extractor)
absa(df, "PD", "en", aspect_extractor)

  0%|          | 0/41 [00:00<?, ?it/s, classifying aspect sentiments...]/usr/local/lib/python3.8/dist-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:410: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]
100%|██████████| 41/41 [01:10<00:00,  1.72s/it, classifying aspect sentiments...]


The results of aspect term extraction have been saved in /content/atepc_inference.result.json


100%|██████████| 6/6 [00:10<00:00,  1.80s/it, classifying aspect sentiments...]


The results of aspect term extraction have been saved in /content/atepc_inference.result.json


100%|██████████| 14/14 [00:22<00:00,  1.57s/it, classifying aspect sentiments...]


...

100%|██████████| 2/2 [00:03<00:00,  1.63s/it, classifying aspect sentiments...]


The results of aspect term extraction have been saved in /content/atepc_inference.result.json
The results of aspect term extraction have been saved in /content/atepc_inference.result.json


100%|██████████| 9/9 [00:15<00:00,  1.71s/it, classifying aspect sentiments...]


The results of aspect term extraction have been saved in /content/atepc_inference.result.json
The results of aspect term extraction have been saved in /content/atepc_inference.result.json


,Aspects,Negative,Neutral,Positive
,count,sum,sum,sum
Aspects,,,,
work,3,3,0,0
use,2,1,0,1
interview,2,0,2,0
system,2,2,0,0
teacher,2,1,0,1
conditions,2,1,0,1
job,2,0,1,1
privileges,2,2,0,0
